# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f7ba6218820>
├── 'a' --> tensor([[-1.2354, -0.2507,  0.4229],
│                   [-1.5643, -0.6036, -0.3413]])
└── 'x' --> <FastTreeValue 0x7f7ba41a1670>
    └── 'c' --> tensor([[ 0.0176,  0.7433, -0.5322, -1.3098],
                        [-0.0846, -1.9894,  1.3061,  0.8924],
                        [ 0.4106, -1.0783, -0.4413,  1.5059]])

In [4]:
t.a

tensor([[-1.2354, -0.2507,  0.4229],
        [-1.5643, -0.6036, -0.3413]])

In [5]:
%timeit t.a

61.6 ns ± 0.0467 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f7ba6218820>
├── 'a' --> tensor([[-1.8065, -1.2873, -0.1833],
│                   [ 1.0365,  1.0622, -2.0919]])
└── 'x' --> <FastTreeValue 0x7f7ba41a1670>
    └── 'c' --> tensor([[ 0.0176,  0.7433, -0.5322, -1.3098],
                        [-0.0846, -1.9894,  1.3061,  0.8924],
                        [ 0.4106, -1.0783, -0.4413,  1.5059]])

In [7]:
%timeit t.a = new_value

64 ns ± 0.0465 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.2354, -0.2507,  0.4229],
               [-1.5643, -0.6036, -0.3413]]),
    x: Batch(
           c: tensor([[ 0.0176,  0.7433, -0.5322, -1.3098],
                      [-0.0846, -1.9894,  1.3061,  0.8924],
                      [ 0.4106, -1.0783, -0.4413,  1.5059]]),
       ),
)

In [10]:
b.a

tensor([[-1.2354, -0.2507,  0.4229],
        [-1.5643, -0.6036, -0.3413]])

In [11]:
%timeit b.a

56.4 ns ± 0.0564 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.1082,  1.0826,  0.4940],
               [-1.4391, -1.1967, -0.0287]]),
    x: Batch(
           c: tensor([[ 0.0176,  0.7433, -0.5322, -1.3098],
                      [-0.0846, -1.9894,  1.3061,  0.8924],
                      [ 0.4106, -1.0783, -0.4413,  1.5059]]),
       ),
)

In [13]:
%timeit b.a = new_value

488 ns ± 0.104 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

858 ns ± 3.66 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.7 µs ± 46.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 676 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 199 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7af6ea7550>
├── 'a' --> tensor([[[-1.2354, -0.2507,  0.4229],
│                    [-1.5643, -0.6036, -0.3413]],
│           
│                   [[-1.2354, -0.2507,  0.4229],
│                    [-1.5643, -0.6036, -0.3413]],
│           
│                   [[-1.2354, -0.2507,  0.4229],
│                    [-1.5643, -0.6036, -0.3413]],
│           
│                   [[-1.2354, -0.2507,  0.4229],
│                    [-1.5643, -0.6036, -0.3413]],
│           
│                   [[-1.2354, -0.2507,  0.4229],
│                    [-1.5643, -0.6036, -0.3413]],
│           
│                   [[-1.2354, -0.2507,  0.4229],
│                    [-1.5643, -0.6036, -0.3413]],
│           
│                   [[-1.2354, -0.2507,  0.4229],
│                    [-1.5643, -0.6036, -0.3413]],
│           
│                   [[-1.2354, -0.2507,  0.4229],
│                    [-1.5643, -0.6036, -0.3413]]])
└── 'x' --> <FastTreeValue 0x7f7af6ea7580>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 66.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7af6ea7cd0>
├── 'a' --> tensor([[-1.2354, -0.2507,  0.4229],
│                   [-1.5643, -0.6036, -0.3413],
│                   [-1.2354, -0.2507,  0.4229],
│                   [-1.5643, -0.6036, -0.3413],
│                   [-1.2354, -0.2507,  0.4229],
│                   [-1.5643, -0.6036, -0.3413],
│                   [-1.2354, -0.2507,  0.4229],
│                   [-1.5643, -0.6036, -0.3413],
│                   [-1.2354, -0.2507,  0.4229],
│                   [-1.5643, -0.6036, -0.3413],
│                   [-1.2354, -0.2507,  0.4229],
│                   [-1.5643, -0.6036, -0.3413],
│                   [-1.2354, -0.2507,  0.4229],
│                   [-1.5643, -0.6036, -0.3413],
│                   [-1.2354, -0.2507,  0.4229],
│                   [-1.5643, -0.6036, -0.3413]])
└── 'x' --> <FastTreeValue 0x7f7af6ea7eb0>
    └── 'c' --> tensor([[ 0.0176,  0.7433, -0.5322, -1.3098],
                        [-0.0846, -1.9894,  1.3061,  0.8924],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 94.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.6 µs ± 140 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.2354, -0.2507,  0.4229],
                [-1.5643, -0.6036, -0.3413]],
       
               [[-1.2354, -0.2507,  0.4229],
                [-1.5643, -0.6036, -0.3413]],
       
               [[-1.2354, -0.2507,  0.4229],
                [-1.5643, -0.6036, -0.3413]],
       
               [[-1.2354, -0.2507,  0.4229],
                [-1.5643, -0.6036, -0.3413]],
       
               [[-1.2354, -0.2507,  0.4229],
                [-1.5643, -0.6036, -0.3413]],
       
               [[-1.2354, -0.2507,  0.4229],
                [-1.5643, -0.6036, -0.3413]],
       
               [[-1.2354, -0.2507,  0.4229],
                [-1.5643, -0.6036, -0.3413]],
       
               [[-1.2354, -0.2507,  0.4229],
                [-1.5643, -0.6036, -0.3413]]]),
    x: Batch(
           c: tensor([[[ 0.0176,  0.7433, -0.5322, -1.3098],
                       [-0.0846, -1.9894,  1.3061,  0.8924],
                       [ 0.4106, -1.0783, -0.4413,  1.5059]],
         

In [26]:
%timeit Batch.stack(batches)

78.8 µs ± 370 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.2354, -0.2507,  0.4229],
               [-1.5643, -0.6036, -0.3413],
               [-1.2354, -0.2507,  0.4229],
               [-1.5643, -0.6036, -0.3413],
               [-1.2354, -0.2507,  0.4229],
               [-1.5643, -0.6036, -0.3413],
               [-1.2354, -0.2507,  0.4229],
               [-1.5643, -0.6036, -0.3413],
               [-1.2354, -0.2507,  0.4229],
               [-1.5643, -0.6036, -0.3413],
               [-1.2354, -0.2507,  0.4229],
               [-1.5643, -0.6036, -0.3413],
               [-1.2354, -0.2507,  0.4229],
               [-1.5643, -0.6036, -0.3413],
               [-1.2354, -0.2507,  0.4229],
               [-1.5643, -0.6036, -0.3413]]),
    x: Batch(
           c: tensor([[ 0.0176,  0.7433, -0.5322, -1.3098],
                      [-0.0846, -1.9894,  1.3061,  0.8924],
                      [ 0.4106, -1.0783, -0.4413,  1.5059],
                      [ 0.0176,  0.7433, -0.5322, -1.3098],
                      [-0.0846, -

In [28]:
%timeit Batch.cat(batches)

144 µs ± 334 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

306 µs ± 433 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
